In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt

py.init_notebook_mode(connected=False)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/glassdoor-analyze-gender-pay-gap/Glassdoor Gender Pay Gap.csv')

**O que eu estou procurando?** *Bom, eu quero saber sobre diferenças de salário entre homens e mulheres*

In [ ]:
df.head() #Dando uma olhadinha no nosso dataset

In [ ]:
df.shape

Nosso dataset tem 9 classes e 1000 instâncias.. **Será que os gêneros estão balanceados?**

In [ ]:
df.describe() # Estatisticas descritivas

Aqui já conseguimos ver alguns dados que são interessantes, como a média de salário (BasePay) e a média de idades (Age). **Notem o desvio padrão da coluna BasePay, é um número beem alto, e isso nos diz que há muita *variação* para ela.**

In [ ]:
femaleDs = df[df['Gender'] == 'Female'].loc[:,['BasePay','JobTitle', 'Education', 'Seniority']]
maleDs = df[df['Gender'] == 'Male'].loc[:,['BasePay','JobTitle', 'Education', 'Seniority']]

In [ ]:
print('Há {} pessoas do sexo feminino e {} do sexo masculino na base de dados'.format(len(femaleDs.index), len(maleDs.index)))
#Wohoo! Está até balanceado esse dataset para nossa analise 

In [ ]:
#Sobre o desvio padrão
print('O desvio padrão da BasePay para o sexo feminino é de: {} e para masculino: {}'.format(np.std(femaleDs.BasePay), np.std(maleDs.BasePay)))

**Vamos ver as diferenças agora de uma forma mais visual**

In [ ]:
def DistPlot(x, color):
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 6)
    sns.distplot(x, bins=100, color=color)

No gráfico de distribuição abaixo, conseguimos visualizar melhor o desvio padrão (std) da nossa coluna salário (BasePay) para o sexo feminino

In [ ]:
DistPlot(femaleDs.BasePay, 'blue')

No gráfico de distribuição abaixo, conseguimos visualizar melhor o desvio padrão (std) da nossa coluna salário (BasePay) para o sexo masculino

In [ ]:
DistPlot(maleDs.BasePay, 'green')

In [ ]:
print('O maior salário para o sexo feminino é de: ${} e para masculino: ${}'.format(femaleDs.BasePay.max(), maleDs.BasePay.max()))

**Parecem números bem altos né? Já sabemos que tem grande variação de salários mas será que esses pontos max podem ser considerados *outliers*?  Hmmm.. vamos ver** 

Caso não saiba... outliers são pontos que representam "exceções na regra", ou seja, pontos com comportamento que não representam nossa distribuição.

Ex: Eu quero saber o comportamento de compra de um cliente que compra todos os dias 1 peixe na vendinha mas em um determinado dia do mês ele comprou 200 peixes por razões não esclarecidas.. **Poxa, se no mês todo ele comprou 1 peixinho e um dia 200 peixes será que eu preciso considerar esses 200?** *Não*, não no caso em que eu estou interessada em saber do comportamento que mais se repete...

In [ ]:
boxFemale = go.Box(y = femaleDs.BasePay,
                name = 'Feminino',
                marker = {'color': '#e74c3c'})
boxMale = go.Box(y = maleDs.BasePay,
                name = 'Masculino',
                marker = {'color': '#00a000'})

data = [boxFemale, boxMale]

layout = go.Layout(title = 'Dispersão de salário por gênero',
                   titlefont = {'family': 'Arial',
                                'size': 22,
                                'color': '#7f7f7f'},
                   xaxis = {'title': 'Gênero'},
                   yaxis = {'title': 'Salário'},
                   paper_bgcolor = 'rgb(243, 243, 243)',
                   plot_bgcolor = 'rgb(243, 243, 243)')


fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

***Eiiita*** Podemos ver que tanto o sexo feminino quanto masculino possuem possiveis outliers (Nesse momento saber que eles existem é o mais importante, não vou me preocupar muito com eles agora). 

**Observação importante:** a maioria da nossa população têm seus salários bem abaixo do max mas notem que o 2º e 3º quartil referente ao sexo masculino está entre 80k e 115k enquanto que para o feminino está entre 73k e 106k, e claro que nossa mediana é mais alta para o masculino. 
**Note também que o maior salário da nossa base é de uma pessoa do sexo MASCULINO.**

Existe uma diferença mesmo que em primeira vista pareça sutil mas vamos continuar. Acho que faria mais sentido para nossa analise levar em consideração Escolaridade e Profissão... 

In [ ]:
sns.catplot(x="Gender", hue="Education", kind="count", data=df)

In [ ]:
df.JobTitle.unique()

In [ ]:
sns.catplot(x="Gender", hue="JobTitle", kind="count", data=df)

Bem, por causa da diferença de profissões presentes no nosso dataset não seria justo comparar os salários para algumas pois não estaria balanceado.. Então vamos nos restringir a Data Scientist e Graphic Designer

In [ ]:
femaleDs = femaleDs.loc[(femaleDs['JobTitle'] == 'Graphic Designer') | (femaleDs['JobTitle'] == 'Data Scientist')]
maleDs = maleDs.loc[(maleDs['JobTitle'] == 'Graphic Designer') | (maleDs['JobTitle'] == 'Data Scientist')]

Bom, não vamos estender muito nossa analise, vamos só dar uma olhadinha nos maiores salários para "Data Scientist" e "Graphic Designer" para ambos os gêneros. 

In [ ]:
femaleDs.nlargest(15, "BasePay") 

In [ ]:
maleDs.nlargest(15, "BasePay") 

Bom, existem coisas estranhas nessa vida... 
O maior salário é de uma pessoa de gênero feminino como Data Scientist possuindo até o High School e com 5 anos de experiência.
*E ainda note que considerando só essas duas profissões, o sexo feminino recebe mais... Confira no boxplot abaixo.*

In [ ]:
boxFemale = go.Box(y = femaleDs.BasePay,
                name = 'Feminino',
                marker = {'color': '#e74c3c'})
boxMale = go.Box(y = maleDs.BasePay,
                name = 'Masculino',
                marker = {'color': '#00a000'})

data = [boxFemale, boxMale]

layout = go.Layout(title = 'Dispersão de salário por gênero',
                   titlefont = {'family': 'Arial',
                                'size': 22,
                                'color': '#7f7f7f'},
                   xaxis = {'title': 'Gênero'},
                   yaxis = {'title': 'Salário'},
                   paper_bgcolor = 'rgb(243, 243, 243)',
                   plot_bgcolor = 'rgb(243, 243, 243)')


fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
print("Por desencargo de consciência, temos {} pessoas do sexo feminino e {} do masculino".format(len(femaleDs.index),len(maleDs.index)))

In [ ]:
sns.catplot(x="Seniority", y="BasePay", hue="JobTitle", data=femaleDs)
sns.catplot(x="Seniority", y="BasePay", hue="JobTitle", data=maleDs)

É isso, pelo dataset podemos ver que no geral pessoas do sexo feminino recebem menos que pessoas do sexo masculino, e que para algumas profissões o cenário pode ser o inverso. Gostaríamos mesmo é que não tivessem desigualdades pelo gênero...

Poderíamos ter explorado muito mais esse conjunto de dados mas termino por aqui... Se você ficou até o final, me dê um feedback pois é muito importante para minha evolução, se achou algo incorreto ou ficou com dúvidas me fala também :D Obrigada 